In [80]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ING Lion's den 2024!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#### Michal Bryzik(*)
#### Kacper Gruca($)
#### Michal Niegierewicz(&)
#### Jan Slusarek(^)

# Zadanie 1 

# Jesli robicie to na VScode ogarnijcie jadro!!!!!

"""""
Plan dzialania: (Jbc to kazdy podaje tu swoje propozycje, jak sie kto z czym np. niezgadza niech napisze w nawiasie)
1. Wczytac dane i szybko je opisac liczebnosc nulle itd. 
2. Ja bym tutaj posortowal zmienne w macierzy tzn. kolumna 1-16 numeryczne a od 17-do konca binarne
3. Wlasciwe czyszczenie danych, elminacja, aproksymacja, asocjacja itd.
4. Statystyki opisowe, wykresy (rozklady, heatmapy, boxploty itd. )
5. Transformacja danych MinMax, standaryzacja itd. (ogolnie manipulacja tak zeby w razie czego logit byl jak najbardziej dojebany)
6. Trenowanie Logita
7. Wybor najlepszego i sprawdzenie z zalozeniami 




"""""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [81]:
data=pd.read_csv('../data/input/development_sample.csv') # Wczytujemy^
data.head() # Pierwsze 5 rekordow^

,ID,customer_id,application_date,target,Application_status,Var1,Var2,Var3,Var4,Var5,...,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,_r_
0,11034977,32537148,01Feb2010 0:00:00,0.0,Approved,1,2.0,1,7800,99,...,1,1,0,6768.42,0.00,0,0,0,3899,0.267045
1,11034978,32761663,01Feb2010 0:00:00,0.0,Approved,1,1.0,2,11100,78,...,0,0,15000,5937.66,3870.60,0,0,0,3899,0.533040
2,11034979,32701063,01Feb2010 0:00:00,0.0,Approved,2,3.0,1,2400,15,...,2,2,0,5647.77,1463.08,0,0,10,3899,0.861555
3,11034980,32386786,01Feb2010 0:00:00,0.0,Approved,3,1.0,2,11800,30,...,0,0,0,3594.90,21563.78,0,0,10,3899,0.506451
4,11034981,32692110,02Feb2010 0:00:00,NaN,Rejected,1,2.0,1,10200,72,...,2,2,0,9908.45,0.00,0,0,20,3899,0.210913


In [82]:
data.shape # Wymiar macierzy danych treningowych^

(50000, 36)

In [83]:
data.isnull().sum() # Zliczamy liczbe nulli dla kazdej kolumny^

ID                        0
customer_id               0
application_date          0
target                13282
Application_status        0
Var1                      0
Var2                   1403
Var3                   1403
Var4                      0
Var5                      0
Var6                      0
Var7                      0
Var8                  29173
Var9                      0
Var10                 37538
Var11                     0
Var12                 37538
Var13                     0
Var14                     0
Var15                     0
Var16                     0
Var17                    40
Var18                 37416
Var19                 29173
Var20                     0
Var21                     0
Var22                     0
Var23                     0
Var24                     0
Var25                 10088
Var26                 19883
Var27                     0
Var28                     0
Var29                     0
Var30                     0
_r_                 

In [84]:
data.info() # Analityka z typami danych dla kazdej kolumny ^ 
# Hmmm pasuje je posortowac kolumnami tzn. najpierw numeryczne potem kategoryczne porzadkowe i na koncu binarne^ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  50000 non-null  int64  
 1   customer_id         50000 non-null  int64  
 2   application_date    50000 non-null  object 
 3   target              36718 non-null  float64
 4   Application_status  50000 non-null  object 
 5   Var1                50000 non-null  int64  
 6   Var2                48597 non-null  float64
 7   Var3                48597 non-null  object 
 8   Var4                50000 non-null  int64  
 9   Var5                50000 non-null  int64  
 10  Var6                50000 non-null  int64  
 11  Var7                50000 non-null  float64
 12  Var8                20827 non-null  float64
 13  Var9                50000 non-null  int64  
 14  Var10               12462 non-null  float64
 15  Var11               50000 non-null  int64  
 16  Var1

## Clean dataset

### Define columns by datetypes and sort df

We define the target type of the column basing on the description file and exploring data

In [85]:
# Define columns
date_columns = ['application_date', 'Var13'] 
numeric_columns = ['ID', 'customer_id', '_r_', 'Var9', 'Var10']  
categorical_columns = []
binary_columns = ['target', 'Application_status'] 

In [86]:
# Sprawdzenie jaka cala reszta ma typ danych 
for column in data.columns:
    if column in date_columns + numeric_columns + binary_columns:
        continue  
    elif column in ['Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var15', 'Var16', 'Var17', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var29', 'Var30']:
        numeric_columns.append(column)
    elif column in ['Var2', 'Var3', 'Var11', 'Var12', 'Var14']:
        categorical_columns.append(column)
    elif column in ['Var18', 'Var19', 'Var27', 'Var28']:
        binary_columns.append(column)

In [87]:
print(date_columns)
print(numeric_columns)
print(categorical_columns)
print(binary_columns)

['application_date', 'Var13']
['ID', 'customer_id', '_r_', 'Var9', 'Var10', 'Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var15', 'Var16', 'Var17', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var29', 'Var30']
['Var2', 'Var3', 'Var11', 'Var12', 'Var14']
['target', 'Application_status', 'Var18', 'Var19', 'Var27', 'Var28']


In [89]:
# kolejnosc sortowania wg ktorych ulozone sa kolumny 
sorted_columns = date_columns + numeric_columns + categorical_columns + binary_columns

# wybór wartosci posortowanych wg. zdefiniowanej przez nas kategorii 
sorted_data = data[sorted_columns]

In [91]:
sorted_data.head()

,application_date,Var13,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,01Feb2010 0:00:00,26Nov2004,11034977,32537148,0.267045,5640,NaN,1,7800,99,...,1,5,NaN,2,0.0,Approved,1.0,NaN,0,0
1,01Feb2010 0:00:00,16Sep1996,11034978,32761663,0.533040,3000,NaN,1,11100,78,...,2,2,NaN,4,0.0,Approved,NaN,1.0,0,0
2,01Feb2010 0:00:00,06May1989,11034979,32701063,0.861555,6240,4300.0,2,2400,15,...,1,5,5.0,1,0.0,Approved,NaN,NaN,0,0
3,01Feb2010 0:00:00,06Dec1987,11034980,32386786,0.506451,4560,2500.0,3,11800,30,...,2,6,5.0,1,0.0,Approved,NaN,0.0,0,0
4,02Feb2010 0:00:00,18May1989,11034981,32692110,0.210913,5160,NaN,1,10200,72,...,1,5,NaN,0,NaN,Rejected,1.0,NaN,0,0


### Handle date variables

In [96]:
# Make a copy of sorted data, from this moment we will work on this dataset
df = sorted_data.copy()

In [93]:
# Print date columns
date_columns

['application_date', 'Var13']

In [100]:
# Change date columns to datetime type
df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')
df['Var13'] = pd.to_datetime(df['Var13'], format='%d%b%Y', errors='coerce')         # Var13 - irregular date format

In [102]:
# Check naN values after transformation
print(df['Var13'].isnull().sum(0))
print(df['application_date'].isnull().sum(0))

784
0


After transformation some records for Var13 are empty, we need to analyze it

In [105]:
df.head()

,application_date,Var13,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,2010-02-01,2004-11-26,11034977,32537148,0.267045,5640,NaN,1,7800,99,...,1,5,NaN,2,0.0,Approved,1.0,NaN,0,0
1,2010-02-01,1996-09-16,11034978,32761663,0.533040,3000,NaN,1,11100,78,...,2,2,NaN,4,0.0,Approved,NaN,1.0,0,0
2,2010-02-01,1989-05-06,11034979,32701063,0.861555,6240,4300.0,2,2400,15,...,1,5,5.0,1,0.0,Approved,NaN,NaN,0,0
3,2010-02-01,1987-12-06,11034980,32386786,0.506451,4560,2500.0,3,11800,30,...,2,6,5.0,1,0.0,Approved,NaN,0.0,0,0
4,2010-02-02,1989-05-18,11034981,32692110,0.210913,5160,NaN,1,10200,72,...,1,5,NaN,0,NaN,Rejected,1.0,NaN,0,0


#### Analyze Var13

In [107]:
# Check indexes where conversion caused 'NaT' values
nan_indices = df['Var13'].isna()

# Compare original values of 'Var13' which after conversion became 'NaT'
original_values_with_nan = data.loc[nan_indices, 'Var13']

original_values_with_nan

5        31Dec9999
56       31Dec9999
67       31Dec9999
86       31Dec9999
95       31Dec9999
           ...    
49510    31Dec9999
49537    31Dec9999
49617    31Dec9999
49635    31Dec9999
49765    31Dec9999
Name: Var13, Length: 784, dtype: object

In [108]:
# Check unique values of 'Var13' which after conversion became 'NaT'
original_values_with_nan.unique()

array(['31Dec9999'], dtype=object)

Summarizing we have 784 cases in Var13 with value '31Dec9999'

We treat them as data error and for this moment leave with NAs

### Create working_months column

Var13 = employment date

working_moths = Var13 - application_date


We would like to calculate the number of working months before applying for credit, which will indicate the employee's duration at the given position from employment to loan application.

NAs from Var13 are replaced with 0

We also want to have this column first

In [115]:
# Create a new colum working_months
df['working_months'] = ((df['application_date'] - df['Var13']) / np.timedelta64(1, 'M')).fillna(0).astype(int)

# Delete columns 'application_date' and 'Var13'
df.drop(['application_date', 'Var13'], axis=1, inplace=True)

In [116]:
df.head()

,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,Var7,...,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28,working_months
0,11034977,32537148,0.267045,5640,NaN,1,7800,99,1,108.73,...,5,NaN,2,0.0,Approved,1.0,NaN,0,0,62
1,11034978,32761663,0.533040,3000,NaN,1,11100,78,1,195.99,...,2,NaN,4,0.0,Approved,NaN,1.0,0,0,160
2,11034979,32701063,0.861555,6240,4300.0,2,2400,15,1,248.34,...,5,5.0,1,0.0,Approved,NaN,NaN,0,0,248
3,11034980,32386786,0.506451,4560,2500.0,3,11800,30,6,3538.68,...,6,5.0,1,0.0,Approved,NaN,0.0,0,0,265
4,11034981,32692110,0.210913,5160,NaN,1,10200,72,3,580.26,...,5,NaN,0,NaN,Rejected,1.0,NaN,0,0,248


Move 'working_months' to the first column

In [119]:
# Create a list of columns without 'working_months'
df_sorted_columns = df.columns.tolist()
df_sorted_columns.remove('working_months')
# Add 'working_months' to the first position
df_sorted_columns.insert(0, 'working_months')
# Reindex columns
df = df[df_sorted_columns]

In [121]:
df.head()

,working_months,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,62,11034977,32537148,0.267045,5640,NaN,1,7800,99,1,...,1,5,NaN,2,0.0,Approved,1.0,NaN,0,0
1,160,11034978,32761663,0.533040,3000,NaN,1,11100,78,1,...,2,2,NaN,4,0.0,Approved,NaN,1.0,0,0
2,248,11034979,32701063,0.861555,6240,4300.0,2,2400,15,1,...,1,5,5.0,1,0.0,Approved,NaN,NaN,0,0
3,265,11034980,32386786,0.506451,4560,2500.0,3,11800,30,6,...,2,6,5.0,1,0.0,Approved,NaN,0.0,0,0
4,248,11034981,32692110,0.210913,5160,NaN,1,10200,72,3,...,1,5,NaN,0,NaN,Rejected,1.0,NaN,0,0


### Handle binary variables

In [122]:
# Check unique values for binary columns
for column in binary_columns:
    print(f"Unique values for column '{column}': {df[column].unique()}")

Unique values for column 'target': [ 0. nan  1.]
Unique values for column 'Application_status': ['Approved' 'Rejected']
Unique values for column 'Var18': [ 1. nan  0.]
Unique values for column 'Var19': [nan  1.  0.]
Unique values for column 'Var27': [0 1]
Unique values for column 'Var28': [0 1]


As we can observe column "Application_status" needs to be modified

In [123]:
# Change value "Approved" to 1 and "Rejected" to 0 in column 'Application_status'
df['Application_status'] = df['Application_status'].replace({'Approved': 1, 'Rejected': 0})

In [124]:
df[['Application_status']].head()

,Application_status
0,1
1,1
2,1
3,1
4,0


### Handle categorical variables

Teraz pora na one hot encoding dla tych zmiennych
ktore okreslamy jako kategoryczne nominalne 
W naszym wypadku bedzie to tak:
- Var2
- Var3 
- Var11
- Var12
- Var14

In [125]:
# Check unique values for binary columns
for column in categorical_columns:
    print(f"Unique values for column '{column}': {df[column].unique()}")

Unique values for column 'Var2': [ 2.  1.  3. nan]
Unique values for column 'Var3': ['1' '2' nan 'Direct' '3' 'Online']
Unique values for column 'Var11': [5 2 6 4 3 1 7]
Unique values for column 'Var12': [nan  5.  2.  4.  6.  7.  3.  1.]
Unique values for column 'Var14': [2 4 1 0 3]


In [128]:
sorted_data_encoded = pd.get_dummies(df, columns=categorical_columns)
sorted_data_encoded.head()

,working_months,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,...,Var12_3.0,Var12_4.0,Var12_5.0,Var12_6.0,Var12_7.0,Var14_0,Var14_1,Var14_2,Var14_3,Var14_4
0,62,11034977,32537148,0.267045,5640,NaN,1,7800,99,1,...,0,0,0,0,0,0,0,1,0,0
1,160,11034978,32761663,0.533040,3000,NaN,1,11100,78,1,...,0,0,0,0,0,0,0,0,0,1
2,248,11034979,32701063,0.861555,6240,4300.0,2,2400,15,1,...,0,0,1,0,0,0,1,0,0,0
3,265,11034980,32386786,0.506451,4560,2500.0,3,11800,30,6,...,0,0,1,0,0,0,1,0,0,0
4,248,11034981,32692110,0.210913,5160,NaN,1,10200,72,3,...,0,0,0,0,0,1,0,0,0,0


In [20]:
# Zliczmy teraz dla ile jest rekordow ktore maja wieksza niz 0 wartosc nulli 
# Ustawienie kolumny 'ID' jako indeksu DataFrame
data_l=data.copy()
data_l.set_index('ID', inplace=True)

# Obliczenie liczby nulli w każdym wierszu i wylistowanie kolumn z nullami
null_analysis_per_id = data_l.isnull().apply(lambda x: pd.Series([x.sum(), list(data_l.columns[x])], index=['Null_Count', 'Null_Columns']), axis=1)

null_analysis_per_id
# Tutaj jest pewien problem
# Bo przy one hot encoding zmienne kategoryczne nominalne ktore nie mialy zadnej wartosci Nan 
# maja z automatu przypisana wartosc 0 dla kazdej kategorii w one hot encoding .... 
# Ponadto kazdy rekord w danych posiada przynajmniej jednego nulla 


,Null_Count,Null_Columns
ID,,
11034977,4,"[Var8, Var10, Var12, Var19]"
11034978,3,"[Var10, Var12, Var18]"
11034979,3,"[Var8, Var18, Var19]"
11034980,1,[Var18]
11034981,5,"[target, Var8, Var10, Var12, Var19]"
...,...,...
11084972,3,"[Var18, Var25, Var26]"
11084973,7,"[target, Var8, Var10, Var12, Var19, Var25, Var26]"
11084974,3,"[Var10, Var12, Var18]"


In [22]:
#3 Wlasciwe czyszczenie danych
"""
Jesli chodzi o te nulle dla zmiennych kategorycznych nominalnych 
Tutaj pojawia sie pewien problem, bo musimy zastosowac aproksymacyjny algorytm
 alternatywy wylacznej. Tzn. jesli juz przy jakiejs kategorii 
 mamy 1 przy reszcie musi byc zero np. nie da 
sie byc dzieckiem i starcem rownoczesnie :) 
Ciezko to zastoswac przed one hot coding bo w tym wypadku metryka euklidesowa 
czy manhattan nie ma sensu. Alternatywa tutaj byla by metryka bitowa tzn. Humminga, ale
python nie obsluguje algorytmow klasyfikacji np. KNN w oparciu o ta metryke 
Mam tutaj pomysl czasochlonny jak cholera, ale dobry 
Dla kazdego nulla w kategorii zastosowac algorytm predykcyjny lasow losowych 
czyli dla kazdej kategori czyli kilku zmiennych binarnych robimy osobna klasyfikacje
Wpisujemy 1 tam gdzie algorytm ma najwyzsza pewnosc a w reszcie zero.
To zostawmy na sam koniec, bo wymaga to zeby pozostale zmienne byly juz w miare kompletne,
a wiec aproksymacje nulli zacznijmy od numerycznych 

"""

 


'\nJa bym tu zaczal od tego bajzlu w tych zdekodowanych \nzmiennych kategor. nomin. ktore nie posiadaja wartosci kategorii. \nTutaj pojawia sie pewien problem, bo musimy zastosowac aproksymacyjny algorytm alternatywy wylacznej.\nTzn. jesli juz przy jakiejs kategorii mamy 1 przy reszcie musi byc zero np. nie da \nsie byc dzieckiem i starcem rownoczesnie :) \n\n'